In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
pd.set_option('display.max_rows', None)

## Read Data

In [4]:
df = pd.read_csv('/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv')
print(df.shape)
df.head(1)

(288, 13)


,RANK,PLAYER,SCHOOL/TEAM,POS,AGE,HT,WING,TIER,STRENGTHS,WEAKNESSES,SUMMARY,TIER_DESCRIP,year
0,1,LaMelo Ball,Illawarra Hawks,G,19,6-6,6-9,5,Everything starts with Ball’s elite-level feel...,"The defense isn’t a sure thing, though, becaus...",Ball should enter the NBA as one of the most c...,All-Star Upside,2020


In [5]:
# Add Strengths and Weaknesses
df['full_text'] = df[['STRENGTHS', 'WEAKNESSES']].apply(lambda x: x[0] + ' ' + x[1], axis=1)
# df['full_text'] = df['WEAKNESSES']
df.head(1)

,RANK,PLAYER,SCHOOL/TEAM,POS,AGE,HT,WING,TIER,STRENGTHS,WEAKNESSES,SUMMARY,TIER_DESCRIP,year,full_text
0,1,LaMelo Ball,Illawarra Hawks,G,19,6-6,6-9,5,Everything starts with Ball’s elite-level feel...,"The defense isn’t a sure thing, though, becaus...",Ball should enter the NBA as one of the most c...,All-Star Upside,2020,Everything starts with Ball’s elite-level feel...


## Preprocessing

In [6]:
df_2023 = df[df['year'] == 2023]

In [7]:
val = df_2023.sample(40, random_state=42)
test = df_2023[~df_2023.index.isin(val.index)]
val.shape, test.shape

((40, 14), (35, 14))

In [8]:
train = df[~df.index.isin(df_2023.index)]\
        .reset_index().drop('index', axis=1)
train.shape

(213, 14)

In [222]:
# # # data augmentation
# train_copy = train.copy()
# train_copy['full_text'] = train_copy[['STRENGTHS', 'WEAKNESSES']].apply(lambda x: x[1] + ' ' + x[0], axis=1)
# train = pd.concat([train, train_copy])\
#           .reset_index().drop('index', axis=1)
# train.shape

In [9]:
train_count_dict = train.groupby('TIER')['PLAYER'].count().to_dict()
train_count_dict

{1: 72, 2: 81, 3: 34, 4: 15, 5: 11}

In [224]:
# max_n = max(train_count_dict.values())

# for i in range(1, 6):
#     diff = max_n - train_count_dict[i]
#     if diff != 0:
#         random_sample = train[train['TIER'] == i].sample(diff, 
#                                                          random_state=42,
#                                                          replace=True)
#         train = pd.concat([train, random_sample])\
#                   .reset_index().drop('index', axis=1)

In [10]:
train.groupby('TIER')['PLAYER'].count().to_dict()

{1: 72, 2: 81, 3: 34, 4: 15, 5: 11}

## Preprocessing


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [12]:
MODEL = 'allenai/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weigh

In [13]:
model

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [17]:
model.classifier = torch.nn.Linear(
    768,
    1
)
# model.classifier = nn.Sequential(
#     nn.Linear(model.classifier.in_features, 512),
#     nn.Dropout(0.2),
#     nn.Linear(512, 128),
#     nn.Dropout(0.2),
#     nn.Linear(128, 1)
# )

In [46]:
def preprocess_function(examples):
    tokens = tokenizer(examples['full_text'], 
                       max_length=512*4)
    if 'TIER' in examples:
        return {**tokens, "labels": [[float(x)] for x in examples['TIER']]}
    else:
        return tokens

In [47]:
train_dataset = (
    Dataset.from_pandas(train[['PLAYER', 'full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
)

val_dataset = (
    Dataset.from_pandas(val[['PLAYER', 'full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
)

test_dataset = (
    Dataset.from_pandas(test[['PLAYER', 'full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [48]:
example = train_dataset[0]
example['PLAYER'], example['labels']

('Xavier Tillman', [2.0])

## Modeling

In [51]:
from sklearn.metrics import mean_squared_error

In [112]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # replace with regression loss
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = F.mse_loss(torch.mean(outputs.logits, 1), labels)
        return (loss, outputs) if return_outputs else loss

In [113]:
os.environ["WANDB_DISABLED"] = "true"
torch.cuda.empty_cache()

In [114]:
CONFIG = {
    "weight_decay": 0.01,
    "num_train_epochs": 20.0,
    "load_best_model_at_end": True,
    "metric_for_best_model": 'eval_loss',
    "lr_scheduler_type": 'cosine_with_restarts',
}

In [115]:
run_name = f"fpell-{int(time.time())}"

print("=" * 50)
print(f"Starting run: {run_name}")
print("=" * 50)

training_args = TrainingArguments(
    run_name=run_name,
    output_dir="./results",
    save_total_limit=5,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    **CONFIG,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
#     compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting run: fpell-1692369836


Step,Training Loss,Validation Loss
50,1.712500,1.540120
100,1.351500,1.303240
150,1.233300,1.296150
200,1.086100,1.791622
250,1.416700,1.383635
300,0.971300,1.325579
350,1.158000,1.306836
400,0.858700,1.428691
450,1.498100,1.304125
500,1.130000,1.324717


TrainOutput(global_step=2140, training_loss=1.1102424461150837, metrics={'train_runtime': 1900.0111, 'train_samples_per_second': 2.242, 'train_steps_per_second': 1.126, 'total_flos': 3360384529086576.0, 'train_loss': 1.1102424461150837, 'epoch': 20.0})

In [116]:
temp = trainer.predict(train_dataset)
mean_squared_error([x[0] for x in temp[1]],
                   [x[0] for x in temp[0]])

1.4585379

In [117]:
df[['TIER', 'TIER_DESCRIP']].drop_duplicates()

,TIER,TIER_DESCRIP
0,5,All-Star Upside
3,4,High-Leverage Starters
11,3,Upside Swings
17,2,Rotation Players
49,1,Second Round and Two-Ways


In [118]:
data = []

for i in range(len(train_dataset)):
    data.append([train_dataset[i]['PLAYER'], 
                 train_dataset[i]['labels'][0],
                 temp[0][i][0]])
    
res = pd.DataFrame(data, columns=['PLAYER', 'TIER', 'pred'])
res['diff'] = res['TIER'] - res['pred']
res.sort_values('diff', ascending=False).drop_duplicates()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7                                                                                             │
│    8 res = pd.DataFrame(data, columns=['PLAYER', 'TIER', 'pred'])                                │
│    9 res['diff'] = res['TIER'] - res['pred']                                                     │
│ ❱ 10 res.sort_values('diff', ascending=False).drop_duplicates()                                  │
│   11                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/util/_decorators.py:331 in wrapper                │
│                                                                                                  │
│   328 │   │   │   │   │   FutureWarning,                                                         │
│   329 │   │   │   │   │   stacklevel=find_stack_level(),                                         │
│   330 │   │   │   │   )                                                                          │
│ ❱ 331 │   │   │   return func(*args, **kwargs)                                                   │
│   332 │   │                                                                                      │
│   333 │   │   # error: "Callable[[VarArg(Any), KwArg(Any)], Any]" has no                         │
│   334 │   │   # attribute "__signature__"                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/frame.py:6672 in drop_duplicates             │
│                                                                                                  │
│    6669 │   │                                                                                    │
│    6670 │   │   inplace = validate_bool_kwarg(inplace, "inplace")                                │
│    6671 │   │   ignore_index = validate_bool_kwarg(ignore_index, "ignore_index")                 │
│ ❱  6672 │   │   duplicated = self.duplicated(subset, keep=keep)                                  │
│    6673 │   │                                                                                    │
│    6674 │   │   result = self[-duplicated]                                                       │
│    6675 │   │   if ignore_index:                                                                 │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/frame.py:6814 in duplicated                  │
│                                                                                                  │
│    6811 │   │   │   result.name = None                                                           │
│    6812 │   │   else:                                                                            │
│    6813 │   │   │   vals = (col.values for name, col in self.items() if name in subset)          │
│ ❱  6814 │   │   │   labels, shape = map(list, zip(*map(f, vals)))                                │
│    6815 │   │   │                                                                                │
│    6816 │   │   │   ids = get_group_index(                                                       │
│    6817 │   │   │   │   labels,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pandas/core/frame.p

In [120]:
temp = trainer.predict(val_dataset)
mean_squared_error([x[0] for x in temp[1]],
                   [x[0] for x in temp[0]])

1.5438061

In [ ]:
data = []

for i in range(len(val_dataset)):
    data.append([val_dataset[i]['PLAYER'], 
                 val_dataset[i]['labels'][0],
                 temp[0][i][0]])
    
res = pd.DataFrame(data, columns=['PLAYER', 'TIER', 'pred'])
res['diff'] = res['TIER'] - res['pred']
res.sort_values('diff', ascending=False)

In [121]:
temp = trainer.predict(test_dataset)
mean_squared_error([x[0] for x in temp[1]],
                   [x[0] for x in temp[0]])

1.3856564

In [119]:
data = []

for i in range(len(test_dataset)):
    data.append([test_dataset[i]['PLAYER'], 
                 test_dataset[i]['labels'][0],
                 temp[0][i][0]])
    
res = pd.DataFrame(data, columns=['PLAYER', 'TIER', 'pred'])
res['diff'] = res['TIER'] - res['pred']
res.sort_values('diff', ascending=False)

,PLAYER,TIER,pred,diff
20,Scoot Henderson,5.0,[1.6395947],[3.3604054]
32,Cam Whitmore,5.0,[1.639632],[3.360368]
29,Taylor Hendricks,4.0,[1.6396939],[2.3603063]
19,Ausar Thompson,4.0,[1.6397212],[2.3602788]
14,Gradey Dick,3.0,[1.6395752],[1.3604248]
3,Jalen Hood-Schifino,3.0,[1.6396581],[1.3603419]
30,Bilal Coulibaly,3.0,[1.6396915],[1.3603085]
26,Colby Jones,2.0,[1.639586],[0.36041403]
17,Olivier-Maxence Prosper,2.0,[1.6396086],[0.36039138]
27,Julian Phillips,2.0,[1.6396137],[0.36038625]
